# Demo: Real Data #1
## for Dynamic Quantum Mapping (DQM)

This notebook shows an example of working with DQM on real data.

The (anonymized) data set is __[RNA-Seq](https://en.wikipedia.org/wiki/RNA-Seq)__ data from human blood samples, each from a different person. The data set has 186 samples and 20,443 dimensions. Each dimension is a measure of expression level in the blood for a single gene.

There are 4 'cohorts' (groups) in the metadata. The cohorts are real, but we won't define them further here. (Pick your meaning of choice: they could be 4 different hospitals, 4 different demographics, 4 different diseases, etc.)

## First, make sure the 'dqm' package is in your PYTHONPATH

(*See the __[full installation instructions](https://dqm.readthedocs.io/en/latest/installation.html)__ if you need them.*)

If it isn't already, you can uncomment the code below and edit as needed to make sure that the 'dqm' package is in your PYTHONPATH.

In [ ]:
# Note: you need the parent folder of the Python 'dqm' folder in the PYTHONPATH. The 'dqm_repository' folder below
# is the outer folder, containing the README file, etc.

#import os, sys
#sys.path.append(os.path.join(os.path.expanduser('~'), 'dqm_repository'))

## Import what we need

In [ ]:
import numpy as np
from dqm import DQM, plot_frames, smooth_frames, get_clusters
import os

# import PyPlot for some basic plotting
try:
    import matplotlib.pyplot as plt
    HAVE_PLT = True
except:
    HAVE_PLT = False
print('Have PyPlot:', HAVE_PLT)

## Load the data set

Note that the data matrix has 186 rows (samples) and 20,443 columns (dimensions).

In [ ]:
# load data and metadata from disk
loaded = np.load(os.path.join('data', 'demo_real_data_1.npz'))
dat = loaded['dat']
cohorts = loaded['cohorts']

num_rows, num_cols = dat.shape
print('Data matrix has {:,} rows (points/samples) and {:,} columns (dimensions)'.format(num_rows, num_cols))
print("Metadata 'cohorts' vector has {:,} entries".format(cohorts.size))

## Look at the distributions of data and metadata

Data: a log2 transformation has already been applied.

Metadata: there are 4 'cohorts' (groups), with unequal sample sizes.

In [ ]:
# look at data distribution
if HAVE_PLT:
    plt.hist(dat.flatten(), bins=100)
    plt.xlabel('log2 of expression level')
    plt.ylabel('count')
    plt.title('Histogram: Log2 of Raw Data')
    plt.show()
else:
    print("oops -- we don't have PyPlot for plotting...")

print()

# look at metadata distribution
print('Metadata distribution')
print('---------------------')
unique, counts = np.unique(cohorts, return_counts=True)
for idx in range(unique.size):
    print('cohort {} has {} samples'.format(unique[idx], counts[idx]))

## Create a DQM instance and store the raw data

In [ ]:
dqm_obj = DQM()
dqm_obj.verbose = True  # default True
dqm_obj.raw_data = dat

print('Raw data stored in DQM instance has shape:', dqm_obj.raw_data.shape)

## Choose how many PCA dimensions to use and create frame 0

Based on the 'elbows' in the two leftmost PCA plots, we choose to work with the first 50 PCA dimensions. The __[`create_frame_0`](https://dqm.readthedocs.io/en/latest/api.html#dqm.DQM.create_frame_0)__ method reports that those dimensions capture 82.2% of the total variance in the data (as also seen in the rightmost PCA plot).

(*You can ignore warnings from PCA about negative values and unnormalized vectors at the machine-precision level. Those warnings may be suppressed altogether at some point...*)

In [ ]:
dqm_obj.run_pca()
dqm_obj.pca_num_dims = 50
dqm_obj.create_frame_0()

print()
print("In the DQM instance, 'frames' (which now stores frame 0) has shape:", dqm_obj.frames.shape)

## Create a cohort color scheme and plot frame 0

Cohort color scheme
* 1 = Orange
* 2 = Black
* 3 = Blue
* 4 = Red

Remember that __[`plot_frames`](https://dqm.readthedocs.io/en/latest/api.html#dqm.utils.plot_frames)__ uses the first 3 columns in the 2nd dimension by default, so we're looking at the first 3 PCA dimensions.

Observations from frame 0:
1. There are already hints that blue (cohort 3) and red (cohort 4) may be more separable, from the other cohorts and from each other, than orange (cohort 1) and black (cohort 2).

In [ ]:
# create cohort color scheme
cohort_colors = np.zeros((num_rows, 3))
cohort_colors[cohorts == 1] = np.array([1, 0.65, 0])  # orange
cohort_colors[cohorts == 2] = np.array([0, 0, 0])  # black
cohort_colors[cohorts == 3] = np.array([0, 0, 1])  # blue
cohort_colors[cohorts == 4] = np.array([1, 0, 0])  # red

plot_frames(dqm_obj.frames, color=cohort_colors, title='Real Data: First 3 PCA Dimensions, Colored by Cohort')

## Look at mean distance between rows

... to get a sense of scale for 'reasonable' values of sigma.

In [ ]:
dqm_obj.estimate_mean_row_distance()

## Run DQM evolution with sigma = 29

With only 186 points, we can use the default 'full' basis (using all points in the basis) and still explore multiple values of sigma fairly quickly.

We settled on a value of sigma = 29 for our first look at the landscape. (You might try some other values as well, to see what changes.)

Observations from this DQM evolution:
1. As frame 0 hinted, blue (cohort 3) and red (cohort 4) are clearly (not perfectly) separable from the other cohorts.
1. Red forms its own subcluster, which then joins the blue cluster after a pause, making clear that cohorts 3 and 4 are also clearly separable from each other. (*The pause is an example of a quantum 'tunneling' event: the red cluster moves very slowly through a barrier/hill in the landscape before joining the blue cluster.*)
1. Red and blue together form a single linear structure (even though they're separable as distinct clusters), which may be of interest when other metadata fields are applied. **This is a good example of DQM helping with the question of how many models you need for your data -- or, alternately, which subset of your data to apply a model to. Here, a conventional regression algorithm may usefully be applied, but it's now clear that it should only be applied to cohorts 3 and 4.**
1. Orange (cohort 1) and black (cohort 2) are clearly higher variance than the other cohorts, since they mostly remain as outliers at this value of sigma.
1. Notice 3 blue points, next to the main group of red points, which join the blue cluster more quickly than the red points they seem to be right next to. Why do points that are near each other behave so differently? Because they're different in the higher dimensions that we can't see here. **This is a good example of how the animation allows us to see information from more than 3 dimensions in this 3D plot.**

In [ ]:
# run DQM evolution
dqm_obj.sigma = 29
dqm_obj.clear_frames()  # in case we run this block multiple times
dqm_obj.build_operators()
dqm_obj.build_frames_auto(500)

print()

# plot smoothed frames
frames = smooth_frames(dqm_obj.frames)
plot_frames(frames, color=cohort_colors, width=800, height=800,
            title='Real Data: {} DQM Dims, Sigma={:.1f}'.format(dqm_obj.pca_num_dims, dqm_obj.sigma))

## Extract distinct blue and red clusters from smoothed frame 45

Clusters are returned from __[`get_clusters`](https://dqm.readthedocs.io/en/latest/api.html#dqm.utils.get_clusters)__ in decreasing order of size.

Cohort freqencies in the 3 biggest clusters (biggest first) are:
* Cluster 0: 27 blue, 7 red, 2 black
* Cluster 1: 21 red
* Cluster 2: 5 orange, 5 black

In [ ]:
# get clusters from smoothed frame 45
clusters, cluster_sizes = get_clusters(frames[:, :, 45], dqm_obj.mean_row_distance / 100)

# create helper function
def report_cohort_frequencies(cohorts, row_nums):
    unique, counts = np.unique(cohorts[row_nums], return_counts=True)
    for idx in range(unique.size):
        cohort = unique[idx]
        print('cohort {}: {} (of {}) samples'.format(cohort, counts[idx], np.sum(cohorts == cohort)))
# end function report_cohort_frequencies

# report on cohort frequencies for the 3 biggest clusters
for cluster_idx in range(3):
    cluster = clusters[cluster_idx]
    print(f'\ncluster {cluster_idx} (n={len(cluster)})')
    print('---------------------------')
    report_cohort_frequencies(cohorts, cluster)

## Extract single blue/red cluster from the last smoothed frame

Again, we'll look at the cohort frequencies in the 3 biggest clusters (biggest first):
* Cluster 0: 27 blue, 29 red, 2 black
* Cluster 1: 5 orange, 5 black
* Cluster 2: 7 black

In [ ]:
# get clusters from last smoothed frame
clusters, cluster_sizes = get_clusters(frames[:, :, -1], dqm_obj.mean_row_distance / 100)

# report on cohort frequencies for the 3 biggest clusters
for cluster_idx in range(3):
    cluster = clusters[cluster_idx]
    print(f'\ncluster {cluster_idx} (n={len(cluster)})')
    print('---------------------------')
    report_cohort_frequencies(cohorts, cluster)

## Build DQM map for cohorts 3 (blue) and 4 (red)

Here we build a new DQM map for cohorts 3 and 4. (Depending on your context, you might choose instead to use the final cluster from the first DQM map, thus excluding a few outliers and including 2 points from cohort 2.)

Based on the PCA plots, we choose to use the first 20 PCA dimensions.

We choose a relatively large value of sigma, 32, which emphasizes the linear structure(s) in the data. (*The largest value of sigma that still separates blue and red is around 25. That sigma=25 map also shows that red points are somewhat higher variance than blue.*)

Observations from this (sigma=32) DQM evolution (smoothed frames 60-80 show the structure most clearly):
1. When observed in isolation like this, the linear structures for blue and red are *not* exactly colinear. The angle is small, though, so a regression applied to both cohorts together may still be useful.
1. The linear structure is somewhat more extended for red than for blue.

In [ ]:
# subselect data and colors to cohorts 3 and 4
row_nums34 = np.where(np.logical_or(cohorts == 3, cohorts == 4))[0]
dat34 = dat[row_nums34, :]
cohort_colors34 = cohort_colors[row_nums34, :]

# set up DQM instance
dqm_obj34 = DQM()
dqm_obj34.raw_data = dat34

# choose how many PCA dimensions to use, and create frame 0
dqm_obj34.run_pca()
dqm_obj34.pca_num_dims = 20
dqm_obj34.create_frame_0()

print()
dqm_obj34.estimate_mean_row_distance()
print()

# run DQM evolution
dqm_obj34.sigma = 32
dqm_obj34.build_operators()
dqm_obj34.build_frames_auto(500)

# plot smoothed frames
frames34 = smooth_frames(dqm_obj34.frames)
plot_frames(frames34, color=cohort_colors34,
            title='Real Data: Cohorts 3 & 4, {} DQM Dims, Sigma={:.1f}'.format(dqm_obj34.pca_num_dims, dqm_obj34.sigma))

## Build DQM map for cohorts 1 (orange) and 2 (black)

Here we build a new DQM map for cohorts 1 and 2. (Depending on your context, you might choose instead to use all points *not* in the final cluster from the first DQM map, thus including a few outliers from cohorts 3 and 4 and excluding 2 points from cohort 2.)

Based on the PCA plots, we choose to use the first 30 PCA dimensions.

We choose a value of 30 for sigma, giving us a landscape that shows us the most that we're likely to learn here about these cohorts, which is not all that much. (You might try some other values as well, to see what changes.)

Observations from this DQM evolution:
1. Some 1-D structures may be forming as the main cluster coalesces, but each one is based on only a few points. Overall, this map is much closer to an 'uninteresting' single cluster. (The 1-D structures are only likely to be interesting if other metadata fields show any of them to have a strong/clear relationship with the metadata.)
1. It looks like orange and black are really not separable at all.

In [ ]:
# subselect data and colors to cohorts 1 and 2
row_nums12 = np.where(np.logical_or(cohorts == 1, cohorts == 2))[0]
dat12 = dat[row_nums12, :]
cohort_colors12 = cohort_colors[row_nums12, :]

# set up DQM instance
dqm_obj12 = DQM()
dqm_obj12.raw_data = dat12

# choose how many PCA dimensions to use, and create frame 0
dqm_obj12.run_pca()
dqm_obj12.pca_num_dims = 30
dqm_obj12.create_frame_0()

print()
dqm_obj12.estimate_mean_row_distance()
print()

# run DQM evolution
dqm_obj12.sigma = 30
dqm_obj12.build_operators()
dqm_obj12.build_frames_auto(500)

# plot smoothed frames
frames12 = smooth_frames(dqm_obj12.frames)
plot_frames(frames12, color=cohort_colors12,
            title='Real Data: Cohorts 1 & 2, {} DQM Dims, Sigma={:.1f}'.format(dqm_obj12.pca_num_dims, dqm_obj12.sigma))